In [129]:
import pandas as pd
import numpy as np

# Read in data files

In [256]:
charts = pd.read_csv('datasets/charts.csv')
# songs_and_artist = charts[['song', 'weeks-on-board', 'artist']].groupby(
#     ['song', 'artist']).max().sort_values(by='weeks-on-board', ascending=False)
charts.reset_index(level=['song', 'artist'], inplace=True)

songs_and_artist =  charts.sample(n=300)
songs_and_artist.head()

,song,artist,weeks-on-board
8927,Fingertips - Pt 2,Little Stevie Wonder,15
29382,Too Comfortable,Future,1
5592,Raining On Sunday,Keith Urban,19
8267,Grapevyne,Brownstone,15
24760,Jessie's Girl,Glee Cast,3


In [128]:
# songs_and_artist = pd.read_csv('datasets/lyrics_and_genre.csv')

# Fetch the corresponding genre for each song from discogs API
## Search for genre and lyrics by artist and name of the song 

In [136]:
# !pip install lyricsgenius
import requests
from lyricsgenius import Genius
from ratelimit import limits, sleep_and_retry

# 30 calls per minute
CALLS = 30
RATE_LIMIT = 60


# Params for genre API
URL = 'https://api.discogs.com/database/search?'
api_key = 'inTtqFRUynYdTaZnVvYz'
api_secret = 'CshyGbZvbkPxHiJuxfuqNEYdAQhqGwGR'

# Params for lyrics API
genius = Genius(
    "7Ise90s_tMVx4jQroYGWIOaskQOTBFiCh3G9J8CtgLKBK65cXk3HNFEVMYNMoGBd")


# Fetch genre information
@sleep_and_retry
@limits(calls=CALLS, period=RATE_LIMIT)
def search(song, artist):
    params = {
        'release_title': song.replace('+', ' '),
        'artist': artist,
        'key': api_key,
        'secret': api_secret
    }
    response = requests.get(URL, params=params)
    # print(response.url)
    return response.json()

songs_and_artist['json_genre'] = songs_and_artist.apply(
    lambda x: search(x.song, x.artist), axis=1)


# Fetch lyrics
lyrics = []
for i in range(len(songs_and_artist)):
    try:
        song = genius.search_song(
            songs_and_artist.iloc[i, 0], songs_and_artist.iloc[i, 1])
        lyrics.append(song.lyrics)
    except:
        lyrics.append('empty')
        pass
songs_and_artist['lyrics'] = lyrics

songs_and_artist.head()


Searching for "Radioactive" by Imagine Dragons...
Done.
Searching for "Blinding Lights" by The Weeknd...
Done.
Searching for "Sail" by AWOLNATION...
Done.
Searching for "I'm Yours" by Jason Mraz...
Done.
Searching for "How Do I Live" by LeAnn Rimes...
Done.
Searching for "Party Rock Anthem" by LMFAO Featuring Lauren Bennett & GoonRock...
Done.
Searching for "Counting Stars" by OneRepublic...
Done.
Searching for "Foolish Games/You Were Meant For Me" by Jewel...
Done.
Searching for "Rolling In The Deep" by Adele...
Done.
Searching for "Before He Cheats" by Carrie Underwood...
Done.
Searching for "Ho Hey" by The Lumineers...
Done.
Searching for "You And Me" by Lifehouse...
Done.
Searching for "I Hope" by Gabby Barrett Featuring Charlie Puth...
Done.
Searching for "Circles" by Post Malone...
Done.
Searching for "Demons" by Imagine Dragons...
Done.
Searching for "Macarena (Bayside Boys Mix)" by Los Del Rio...
No results found for: 'Macarena (Bayside Boys Mix) Los Del Rio'
Searching for "Nee

,song,artist,weeks-on-board,json_genre,lyrics
0,Radioactive,Imagine Dragons,87,"{'pagination': {'page': 1, 'pages': 1, 'per_pa...",[Intro]\nWhoah-oh\nWhoah-oh\nWhoah-oh\nWhoah\n...
1,Blinding Lights,The Weeknd,83,"{'pagination': {'page': 1, 'pages': 2, 'per_pa...",[Intro]\nYeah\n\n[Verse 1]\nI've been tryna ca...
2,Sail,AWOLNATION,79,"{'pagination': {'page': 1, 'pages': 1, 'per_pa...",[Intro]\nSail\n\n[Verse 1]\nThis is how I show...
3,I'm Yours,Jason Mraz,76,"{'pagination': {'page': 1, 'pages': 1, 'per_pa...",[Verse 1]\nWell you done done me in; you bet I...
4,How Do I Live,LeAnn Rimes,69,"{'pagination': {'page': 1, 'pages': 1, 'per_pa...",[Verse 1]\nHow do I get through one night with...


In [137]:
# Export dataframe to csv file
songs_and_artist.to_csv('datasets/lyrics_and_genre.csv')

## Select genre based on query result

In [169]:
import re


def func(Query):
    Q = re.findall(r"'genre': \['(\w+)'\]", str(Query))
    return Q


def counter(row):
    if len(row) > 0:
        return max(set(row), key=row.count)
    else:
        return 'None'
        # return dict.C[0]


# Append a genres column that contain the name of the genres based on query result
songs_and_artist['genres'] = songs_and_artist.apply(
    lambda row: func(row['json_genre']), axis=1)

# Store the most frequent genre in the genre column
songs_and_artist['genre'] = songs_and_artist.apply(
    lambda row: counter(row['genres']), axis=1)

# songs_and_artist.drop(['json_genre','genres'], axis=1, inplace=True)
songs_and_artist.head()


,song,artist,weeks-on-board,json_genre,lyrics,genres,genre
0,Radioactive,Imagine Dragons,87,"{'pagination': {'page': 1, 'pages': 1, 'per_pa...",[Intro]\nWhoah-oh\nWhoah-oh\nWhoah-oh\nWhoah\n...,"[Rock, Rock, Rock, Rock, Rock, Rock]",Rock
1,Blinding Lights,The Weeknd,83,"{'pagination': {'page': 1, 'pages': 2, 'per_pa...",[Intro]\nYeah\n\n[Verse 1]\nI've been tryna ca...,"[Electronic, Electronic, Electronic, Pop, Pop]",Electronic
2,Sail,AWOLNATION,79,"{'pagination': {'page': 1, 'pages': 1, 'per_pa...",[Intro]\nSail\n\n[Verse 1]\nThis is how I show...,[Electronic],Electronic
3,I'm Yours,Jason Mraz,76,"{'pagination': {'page': 1, 'pages': 1, 'per_pa...",[Verse 1]\nWell you done done me in; you bet I...,"[Pop, Rock, Electronic, Rock, Reggae]",Rock
4,How Do I Live,LeAnn Rimes,69,"{'pagination': {'page': 1, 'pages': 1, 'per_pa...",[Verse 1]\nHow do I get through one night with...,"[Electronic, Electronic, Pop, Electronic, Pop,...",Electronic


# Create NLP model that predicts the genre of the song based on lyrics and artist

In [170]:
# drop songs of which lyrics can't be found  
data = songs_and_artist[songs_and_artist['lyrics'] != 'empty'] 

# drop songs of which genre can be found
data = songs_and_artist[songs_and_artist['genre'] != 'None'] 


data.describe()

,weeks-on-board
count,190.000000
mean,47.173684
std,8.091170
min,39.000000
25%,41.000000
50%,45.000000
75%,52.000000
max,87.000000


In [251]:
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import StratifiedShuffleSplit


seed = 42
features = ['artist', 'lyrics', 'genre']
test_size = 0.2

# preprocess data 
# Reggae genre has only two records so drop it
data_copy = data[(data['genre']!= 'Reggae') & (data['genre']!= 'Jazz')][features]

# preprocess lyrics
data_copy['lyrics'] = data_copy['lyrics'].apply(lambda x: re.sub(r'(\[.*?\]|\n)', ' ', x))



# train test split
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=test_size, random_state=seed)

sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=seed)
for train_idx, test_idx in sss.split(data_copy, data_copy['genre']):
    strat_train_set = data_copy.iloc[train_idx]
    strat_test_set = data_copy.iloc[test_idx]

print(strat_train_set.shape, strat_test_set.shape)

X_train = strat_train_set.drop('genre', axis=1)
y_train = strat_train_set['genre'].copy()

X_test = strat_test_set.drop('genre', axis=1)
y_test = strat_test_set['genre'].copy()



(149, 3) (38, 3)


In [253]:
y_test

106          Rock
249    Electronic
232          Rock
10           Rock
216    Electronic
264    Electronic
262          Rock
260    Electronic
44           Rock
131           Pop
180          Rock
120          Rock
108           Pop
31           Rock
93           Rock
155    Electronic
287          Rock
192    Electronic
196           Pop
213           Pop
211           Pop
127    Electronic
81           Rock
27           Rock
214    Electronic
37           Rock
219           Pop
116    Electronic
235    Electronic
4      Electronic
70            Pop
206          Rock
25            Pop
87            Pop
167           Pop
36           Rock
6             Pop
34            Pop
Name: genre, dtype: object

In [254]:

# vectorize lyrics and vectorizer
col_transformer = ColumnTransformer([
    ('lyrics_vec', TfidfVectorizer(stop_words="english", analyzer='word',
                                   ngram_range=(1, 2), max_df=1.0,
                                   min_df=1, max_features=None), 'lyrics'),
    ('artist_vec', TfidfVectorizer(stop_words="english", analyzer='word',
                                   ngram_range=(1, 2), max_df=1.0,
                                   min_df=1, max_features=None), 'artist')
])

# fit then transform lyrics and artist name column
X_train = col_transformer.fit_transform(X_train)
X_test = col_transformer.fit_transform(X_test)


# from https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

names = ["Nearest Neighbors",
         # "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "AdaBoost1"]

classifiers = [
    KNeighborsClassifier(2),
    # GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    AdaBoostClassifier(base_estimator=RandomForestClassifier(max_depth=5, n_estimators=20), \
                       learning_rate=0.1, n_estimators=10)
]


for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # evaluate predictions
    accuracy = clf.score(X_test, y_test)
    print("%s Accuracy: %.2f%%" % (name, accuracy * 100.0))


# # evaluate
# print('Accuracy Score Train: ', pipe.score(X_train, y_train))
# print('Accuracy Score Test', pipe.score(X_test, y_test))
# print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
# print('Classification Report: \n', classification_report(y_test, y_pred))


ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 4112 while Y.shape[1] == 23870

Final:  genre + singer + lyric + song name -> Weeks-on-board 

In [185]:
import re


features = ['artist',
            # 'lyrics',
            'genre',
            # 'song'
            ]

# Define X and y
X = data[features]
y = data['weeks-on-board']

# preprocess lyrics
# X['lyrics'] = X['lyrics'].apply(lambda x: re.sub(r'(\[.*?\]|\n)', ' ', x))

# transform text columns with vectorizer
col_transformer = ColumnTransformer([
    # ('lyrics_vec', TfidfVectorizer(stop_words="english", analyzer='word',
    #                                ngram_range=(1, 1), max_df=1.0,
    #                                min_df=1, max_features=None), 'lyrics'),
    ('artist_vec', TfidfVectorizer(stop_words="english", analyzer='word',
                                   ngram_range=(1, 1), max_df=1.0,
                                   min_df=1, max_features=None), 'artist'),
    ('genre_vec', TfidfVectorizer(stop_words="english", analyzer='word',
                                  ngram_range=(1, 1), max_df=1.0,
                                  min_df=1, max_features=None), 'genre'),
    # ('song_vec', TfidfVectorizer(stop_words="english", analyzer='word',
    #                                ngram_range=(1, 1), max_df=1.0,
    #                                min_df=1, max_features=None), 'artist'),
])
X = col_transformer.fit_transform(X)


# split train and test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=seed)

# sss = 


names = ["Nearest Neighbors",
         # "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "AdaBoost1"]

classifiers = [
    KNeighborsClassifier(2),
    # GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    AdaBoostClassifier(base_estimator=RandomForestClassifier(max_depth=5, n_estimators=20), \
                       learning_rate=0.1, n_estimators=10)
]


for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # evaluate predictions
    accuracy = clf.score(X_test, y_test)
    print("%s Accuracy: %.2f%%" % (name, accuracy * 100.0))


Nearest Neighbors Accuracy: 10.53%
Decision Tree Accuracy: 7.89%
Random Forest Accuracy: 7.89%
Neural Net Accuracy: 10.53%
AdaBoost Accuracy: 7.89%
AdaBoost1 Accuracy: 7.89%


In [115]:
from sklearn.metrics import plot_confusion_matrix , classification_report
print(classification_report(y_test,y_pred))
confusion_matrix(y_test, y_pred)

              precision    recall  f1-score   support

  Electronic       0.40      0.15      0.22        13
         Pop       0.42      0.50      0.45        10
        Rock       0.52      0.73      0.61        15

    accuracy                           0.47        38
   macro avg       0.45      0.46      0.43        38
weighted avg       0.45      0.47      0.44        38



array([[ 2,  5,  6],
       [ 1,  5,  4],
       [ 2,  2, 11]])